In [ ]:
import os

import numpy as np

import graph_state.graph_state as gs
from experiment_module.experiment_results_processor import (
    load_bsqn_fidelity_estimation_data, load_multiruns_diagonal_data,
    load_scalability_data)
from experiment_module.experiment_runner import (
    bell_diagonal_scalability_experiment, bell_sampling_diagonal_experiment,
    bell_sampling_fidelity_experiment, partial_tomo_experiment_parallelized)

In [ ]:
"""
We generate the data for 8 qubit graph here.
    1. vary from F=0.5 to F=1 with N = 10^4
    2. vary from 10^2 to 10^5 with F = 0.9 (plotting later)
    X-axis will be log scale so we need to do log scaling X-axis (N)
"""

g = gs.GraphState(8, "complete")
shots = 10_000
# err_model = "single-qubit-dephasing"
err_model = "depolarizing"
repeat = 1000
OVERWRITE = False

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/dge_group_nonoverlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/dge_group_nonoverlapped",
    overwrite=False,
)

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/dge_group_overlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/dge_group_overlapped",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=0.9,
    num_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/bsqn",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    num_shots=shots,
    num_repeats=repeat,
    output_dir="simulation_data/depolarizing/bsqn",
    overwrite=False,
)

# scalability experiment
partial_tomo_experiment_parallelized(
        graphs=[gs.GraphState(n, "complete") for n in range(2, 16)],
        err_model='depolarizing',
        fidelity=0.9,
        total_shots=2 * 10_000,
        overlap_observables=True,
        num_repeats=25,
        output_dir="simulation_data/scalability/dge_group_overlapped",
        overwrite=False,
    )
bell_diagonal_scalability_experiment(
    [gs.GraphState(n, "complete") for n in range(2, 24)],
    'depolarizing',
    fidelity=0.9,
    num_shots=20_000,
    num_repeats=25,
    output_dir="simulation_data/scalability/bsqn",
    overwrite=False,
)

In [ ]:
graphs = [gs.GraphState(i, 'complete') for i in range(21, 201)]
shots = 5_000
err_models = ['depolarizing', 'single-qubit-dephasing', 'bimodal']
stab_factors = '2n'
repeat = 50
OVERWRITE = False

# testing scalability of fidelity estimation
bell_sampling_fidelity_experiment(
    graphs=graphs,
    err_model=err_models,
    fidelity=0.53,
    num_shots=shots,
    num_repeats=repeat,
    stabilizer_factors=stab_factors,
    output_dir='simulation_data/fidelity-estimation/scalability',
    overwrite=False,
    )

# testing varying stabs
num_qubits = 40
shots = 1_000_000
repeat = 50
stabilizer_counts = np.logspace(np.log10(num_qubits), np.log10(num_qubits**3), 50).astype(int)
stab_factors = list(map(str, stabilizer_counts))
bell_sampling_fidelity_experiment(
    graphs=gs.GraphState(40),
    err_model=err_models,
    fidelity=0.53,
    num_shots=shots,
    num_repeats=repeat,
    stabilizer_factors=stab_factors,
    output_dir='simulation_data/fidelity-estimation/varying_stabilizer',
    overwrite=False,
    )

# testing varying shots
num_qubits = 40
shots = np.logspace(2, 5, 50).astype(int)
stab_factors = '2n'
bell_sampling_fidelity_experiment(
    graphs=gs.GraphState(40),
    err_model=err_models,
    fidelity=0.53,
    num_shots=shots,
    num_repeats=repeat,
    stabilizer_factors=stab_factors,
    output_dir='simulation_data/fidelity-estimation/varying_shots',
    overwrite=False,
    )

## Now we process the raw data into CSV files keeping only the relevant information for plotting and archivals

In [ ]:
# Create zip file for experiment 1 and 2 for plotting for 
# "Performance comparison between BSQN and DGE for an 8-node complete graph state"

dge_df = load_multiruns_diagonal_data('simulation_data/depolarizing/dge_group_overlapped', 'tomo', 'DGE')
bsqn_df = load_multiruns_diagonal_data('simulation_data/depolarizing/bsqn', 'bell_diag', 'BSQN')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
dge_df.to_csv(f'{output_dir}/performance_analysis_dge_overlapped.zip', encoding='utf-8', index=False)
bsqn_df.to_csv(f'{output_dir}/performance_analysis_bsqn.zip', encoding='utf-8', index=False)

# Appendix: Create zip file for experiment 1 and 2 for plotting for 
# "Performance comparison between BSQN and DGE for an 8-node complete graph state" for 2 variants of DGEs

dge_nonoverlapped_df = load_multiruns_diagonal_data('simulation_data/depolarizing/dge_group_nonoverlapped', 'tomo', 'DGE-Disjoint')

output_dir = 'simulation_data/processed_csv'
dge_nonoverlapped_df.to_csv(f'{output_dir}/appendix_performance_analysis_dge_non_overlapped.zip', encoding='utf-8', index=False)

In [ ]:
# Create zip file for scalability experiment
# "Performance comparison between BSQN and DGE for an 2 to 200 nodes"

dge_df = load_multiruns_diagonal_data('simulation_data/scalability/dge_group_overlapped', 'tomo', 'DGE')
bsqn_df = load_scalability_data('simulation_data/scalability/bsqn', 'bell_scalability', 'BSQN')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
dge_df.to_csv(f'{output_dir}/scalability_analysis_dge_overlapped.zip', encoding='utf-8', index=False)
bsqn_df.to_csv(f'{output_dir}/scalability_analysis_bsqn.zip', encoding='utf-8', index=False)

In [ ]:
# Create zip file for
# Fidelity estimation data via random sampling with BSQN

df = load_bsqn_fidelity_estimation_data('simulation_data/bell_fidelity_data')
output_dir = 'simulation_data/processed_csv'

os.makedirs(output_dir, exist_ok=True)
df.to_csv(f'{output_dir}/random_sampling_fidelity_estimation_bsqn.zip', encoding='utf-8', index=False)